In [24]:
import os
import xml.etree.ElementTree as ET
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense, Reshape
from tensorflow.keras.models import Model
import cv2
import numpy as np

# Constants
MAX_OBJECTS_PER_IMAGE = 20
NUM_CLASSES = 20  
img_width, img_height = 300, 300  

# Read and convert YOLOv8 annotations to SSD format
def read_yolo_annotation(file_path, img_width, img_height):
    annotations = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            center_x = float(parts[1]) * img_width
            center_y = float(parts[2]) * img_height
            width = float(parts[3]) * img_width
            height = float(parts[4]) * img_height
            xmin = int(center_x - width / 2)
            ymin = int(center_y - height / 2)
            xmax = int(center_x + width / 2)
            ymax = int(center_y + height / 2)
            annotations.append([class_id, xmin, ymin, xmax, ymax])
    return annotations

def create_xml_annotation(img_filename, img_width, img_height, annotations, output_dir):
    annotation = ET.Element("annotation")
    ET.SubElement(annotation, "filename").text = img_filename

    size = ET.SubElement(annotation, "size")
    ET.SubElement(size, "width").text = str(img_width)
    ET.SubElement(size, "height").text = str(img_height)
    ET.SubElement(size, "depth").text = "3"

    for ann in annotations:
        class_id, xmin, ymin, xmax, ymax = ann
        obj = ET.SubElement(annotation, "object")
        ET.SubElement(obj, "name").text = str(class_id)
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = "0"
        ET.SubElement(obj, "difficult").text = "0"

        bndbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bndbox, "xmin").text = str(xmin)
        ET.SubElement(bndbox, "ymin").text = str(ymin)
        ET.SubElement(bndbox, "xmax").text = str(xmax)
        ET.SubElement(bndbox, "ymax").text = str(ymax)

    tree = ET.ElementTree(annotation)
    output_path = os.path.join(output_dir, img_filename.replace('.jpg', '.xml'))
    tree.write(output_path)

def convert_dataset(yolo_dir, img_dir, output_dir, img_width, img_height):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for yolo_file in os.listdir(yolo_dir):
        if yolo_file.endswith('.txt'):
            img_filename = yolo_file.replace('.txt', '.jpg')
            yolo_path = os.path.join(yolo_dir, yolo_file)
            annotations = read_yolo_annotation(yolo_path, img_width, img_height)
            create_xml_annotation(img_filename, img_width, img_height, annotations, output_dir)

# Convert train and val sets
convert_dataset('Annotated_clean/Annotated/labels/train', 'Annotated_clean/Annotated/images/train', 'Annotated_clean/Annotated/annotations/train', img_width, img_height)
convert_dataset('Annotated_clean/Annotated/labels/val', 'Annotated_clean/Annotated/images/val', 'Annotated_clean/Annotated/annotations/val', img_width, img_height)

# Load and preprocess dataset
def load_data(img_dir, ann_dir, img_width, img_height):
    images = []
    annotations = []
    for img_file in os.listdir(img_dir):
        if img_file.endswith('.jpg'):
            img_path = os.path.join(img_dir, img_file)
            image = cv2.imread(img_path)
            image = cv2.resize(image, (img_width, img_height))
            images.append(image)
            
            ann_file = img_file.replace('.jpg', '.xml')
            ann_path = os.path.join(ann_dir, ann_file)
            tree = ET.parse(ann_path)
            root = tree.getroot()
            annots = []
            for obj in root.findall('object'):
                class_id = int(obj.find('name').text)
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)
                annots.append([class_id, xmin, ymin, xmax, ymax])
            annotations.append(annots)
    return np.array(images), annotations

train_images, train_annotations = load_data('Annotated_clean/Annotated/images/train', 'Annotated_clean/Annotated/annotations/train', img_width, img_height)
val_images, val_annotations = load_data('Annotated_clean/Annotated/images/val', 'Annotated_clean/Annotated/annotations/val', img_width, img_height)

# Preprocess annotations
def preprocess_annotations(annotations, img_width, img_height, max_objects, num_classes):
    class_targets = np.zeros((len(annotations), max_objects), dtype=np.int32)
    bbox_targets = np.zeros((len(annotations), max_objects, 4), dtype=np.float32)
    
    for i, ann in enumerate(annotations):
        classes = np.zeros((max_objects,), dtype=np.int32)
        bboxes = np.zeros((max_objects, 4), dtype=np.float32)
        
        for j, obj in enumerate(ann[:max_objects]):
            class_id, xmin, ymin, xmax, ymax = obj
            classes[j] = class_id
            bboxes[j] = [xmin / img_width, ymin / img_height, xmax / img_width, ymax / img_height]
        
        class_targets[i] = classes
        bbox_targets[i] = bboxes
    
    return class_targets, bbox_targets

train_class_targets, train_bbox_targets = preprocess_annotations(train_annotations, img_width, img_height, MAX_OBJECTS_PER_IMAGE, NUM_CLASSES)
val_class_targets, val_bbox_targets = preprocess_annotations(val_annotations, img_width, img_height, MAX_OBJECTS_PER_IMAGE, NUM_CLASSES)

# Define SSD model
def ssd_model(input_shape, num_classes, max_objects):
    input_tensor = Input(shape=input_shape)
    
    # Feature extractor (simplified)
    x = Conv2D(32, (3, 3), activation='relu')(input_tensor)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    
    class_predictions = Dense(max_objects * num_classes, activation='softmax')(x)
    class_predictions = Reshape((max_objects, num_classes), name='class_predictions')(class_predictions)
    
    bbox_predictions = Dense(max_objects * 4, activation='sigmoid')(x)
    bbox_predictions = Reshape((max_objects, 4), name='bbox_predictions')(bbox_predictions)

    model = Model(inputs=input_tensor, outputs=[class_predictions, bbox_predictions])
    return model

input_shape = (img_width, img_height, 3)  # Input shape

model = ssd_model(input_shape, NUM_CLASSES, MAX_OBJECTS_PER_IMAGE)
model.compile(optimizer='adam', 
              loss={'class_predictions': 'sparse_categorical_crossentropy', 'bbox_predictions': 'mse'},
              metrics={'class_predictions': 'accuracy', 'bbox_predictions': 'mse'})

# Train the model
model.fit(train_images, {'class_predictions': train_class_targets, 'bbox_predictions': train_bbox_targets},
          validation_data=(val_images, {'class_predictions': val_class_targets, 'bbox_predictions': val_bbox_targets}),
          epochs=5)


Epoch 1/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 499s 8s/step - bbox_predictions_mse: 0.3673 - class_predictions_accuracy: 0.9255 - loss: 3.3263 - val_bbox_predictions_mse: 0.2950 - val_class_predictions_accuracy: 1.0000 - val_loss: 3.1409
Epoch 2/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 327s 6s/step - bbox_predictions_mse: 0.3019 - class_predictions_accuracy: 1.0000 - loss: 3.1478 - val_bbox_predictions_mse: 0.2949 - val_class_predictions_accuracy: 1.0000 - val_loss: 3.1409
Epoch 3/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 343s 6s/step - bbox_predictions_mse: 0.3012 - class_predictions_accuracy: 1.0000 - loss: 3.1471 - val_bbox_predictions_mse: 0.2949 - val_class_predictions_accuracy: 1.0000 - val_loss: 3.1409
Epoch 4/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 282s 5s/step - bbox_predictions_mse: 0.3017 - class_predictions_accuracy: 1.0000 - loss: 3.1477 - val_bbox_predictions_mse: 0.2949 - val_class_predictions_accuracy: 1.0000 - val_loss: 3.1409
Epoch 5/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 248s 4s/step - bbox_predictions_mse: 0.3017 - class

In [25]:
# Validate and output performance data
performance = model.evaluate(val_images, {'class_predictions': val_class_targets, 'bbox_predictions': val_bbox_targets})
print(f"Validation Loss: {performance[0]}")
print(f"Validation Class Accuracy: {performance[2]}")
print(f"Validation BBox MSE: {performance[1]}")


14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 494ms/step - bbox_predictions_mse: 0.2913 - class_predictions_accuracy: 1.0000 - loss: 3.1372
Validation Loss: 3.1408536434173584
Validation Class Accuracy: 1.0
Validation BBox MSE: 0.2949081361293793


**1.  Setting Up and Data Handling:**

* It defines constants like the maximum number of objects per image (`MAX_OBJECTS_PER_IMAGE`), number of classes (`NUM_CLASSES`), and image width and height (`img_width`, `img_height`).
* It has functions to:
    * Read YOLOv8 annotations (a different object detection format) and convert them to SSD format (`read_yolo_annotation`).
    * Create an XML annotation file for an image containing bounding boxes for detected objects (`create_xml_annotation`).
    * Convert a whole dataset (images and annotations) from YOLOv8 format to SSD format (`convert_dataset`).
* It also defines functions to: 
    * Load and preprocess images (`load_data`). This resizes images and parses the corresponding XML annotations. 
    * Preprocess annotations to a fixed format suitable for training the model (`preprocess_annotations`). 

**2. Building the SSD Model:**

* It defines a function `ssd_model` that builds the SSD model architecture.
  * This is a simplified example with only a few convolutional layers (`Conv2D`), pooling layers (`MaxPooling2D`), a fully connected layer (`Dense`), and reshaping layers (`Reshape`).
  * The model has two outputs:
      * Class predictions: This predicts the class probabilities for each object (one-hot encoded with `softmax`).
      * Bounding box predictions: This predicts the offsets for bounding boxes relative to the image size (using sigmoid activation).

**3. Training and Evaluation:**

* It defines the model input shape based on the image dimensions.
* It creates an SSD model instance with the defined architecture.
* It compiles the model with the Adam optimizer, a combination of loss functions for classification (categorical crossentropy) and bounding box prediction (mean squared error), and accuracy and mean squared error metrics for both outputs.
* It trains the model on the preprocessed training images and annotations (`train_images`, `train_class_targets`, `train_bbox_targets`).
* It evaluates the model on the validation set (`val_images`, `val_class_targets`, `val_bbox_targets`) and prints the validation loss, class accuracy, and bounding box mean squared error.

**Overall, this code snippet demonstrates how to convert annotations from a different format (YOLOv8) to SSD format, preprocess data for training, build a basic SSD model architecture, train it on a dataset, and evaluate its performance.**
